# Introduction to hypothesis testing

Let's say you have a large bag with red, green, and blue balls, and you stick your hand in the bag and draw a dozen balls out. All the balls in the large bag is what we call the **population** and the dozen balls we drew are a **sample**. **Probability** is concerned with what our draw looks like based on how many balls of each color we have in the large bag. In other words, probability asks, given what we have in the population, what should the sample look like. **Statistics** asks the opposite: Given what color balls we drew, what can we say about the composition of balls in the population. In other words, given what we see in the sample, what can we conclude about the population? Drawing conclusions about the population based on the sample is called **statistical inference**.

At the conclusion of this lesson you should be able apply basic classical hypothesis tests to some common situations. In addition to the usual libraries, we will also be using `statsmodels` and `scipy` for many of our examples. Let's begin by loading all the libraries. In this notebook we cover the following examples of tests of hypothesis:

- Two sample tests for continuous variables
- Two sample tests for categorical or count data
- Test for distributions

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(rc = {'figure.figsize': (8, 12)})
import matplotlib.pyplot as plt

import scipy
import scipy.stats as ss
import numpy.random as nr
import statsmodels
import statsmodels.stats as st
import statsmodels.api as sm
import statsmodels.stats.power as sp
import statsmodels.stats.weightstats as ws
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols

We will use a dataset derived from Vladimir Belyaev's [steamdb database](https://github.com/leinstay/steamdb) of all the games offered on the Steam platform to look at various examples of statistical tests. The curated subset of games was restricted to those with complete information and high popularity. It has the following columns:
- name: video game name 
- store_uscore - store.steampowered.com user's score
- igdb_uscore  - igdb.com critic's score
- meta_score - metacritic.com critic's score
- gfq_rating - gamefaqs.gamespot.com user ratings
- windows_only: Available for Windows OS exclusively, or also on Mac / Linux
    - 0: Available on other OS
    - 1: Windows OS exclusive
- price: December 2022 price on steampowered.com
    - 1: < $19.99
    - 2: $19.99 - $24.99
    - 3: > $24.99
- difficulty: gamefaqs.gamespot.com difficulty rating
    - 1: Simple / Easy
    - 2: Just Right
    - 3: Tough / Unforgiving
- time: howlongtobeat.com main story completion time in hours
    - 1: <= 6 hours
    - 2: 7 - 10 hours
    - 3: 11 - 20 hours
    - 4: 21+ hours
- popularity: igdb.com popularity rating. A minimum score of 5 was used to eliminate unpopular games.
    - 1: popularity 5-25
    - 2: popularity > 25

In [ ]:
scores = pd.read_json('../../data/steamdb_popular.json')
scores.head()

### Student's t-test for comparing averages

The difference in means between two normal distributions with unknown variance follows a t-distribution. The t-test is one of the oldest and most widely used classical hypothesis tests. It can be used to

- test whether a population mean has a specified value (**one-sample** t-test).
- test the difference between two means  of two samples (with equal, or unequal variances) (**two-sample** t-test)
- test a paired-response difference from zero, e.g., a before/after drug treatment on patients (**paired** t-test)
- test whether the slope of a line is not zero (covered later in class)
- test the importance of variables (covered later in class)

As a first example, compare the average metacritic score for windows only releases vs multi-platform games:

In [ ]:
windows = scores.loc[scores['windows_only'] == 1, 'meta_score']
multi_platform = scores.loc[scores['windows_only'] == 0, 'meta_score']

bins = [x for x in range(20,101,5)]

sns.displot(data = scores,
            x = 'meta_score', 
            hue = scores['windows_only'], 
            bins = bins, 
            palette = {0:'red', 1:'blue', 'desat':0.5},
            kde = True,
            aspect = 1.75)

plt.title('Distribution of metacritic scores by OS platform availability')
plt.axvline(windows.mean(), 0, color = 'blue', linestyle = '--')
plt.axvline(multi_platform.mean(), 0, color = 'red', linestyle = '--');

Notice that the values of the distributions of these populations overlap quite a bit and the means are close. The question is, are these differences significant?  

You can now compute the two-sample t-test to determine if the difference of means is significant. A number of summary statistics are computed and printed for the test. The two-sided t-test is used to determine if we can reject the **null hypothesis that the difference of means is not significant**.

We can try to implement the t-test manually based on what is described [in this Wikipedia article](https://en.wikipedia.org/wiki/Student%27s_t-test):

In [ ]:
n_win, n_multi = windows.size, multi_platform.size
mean_win, mean_multi = windows.mean(), multi_platform.mean()
var_win, var_multi = windows.var(), multi_platform.var()
var_P = var_win / n_win + var_multi / n_multi

t_stat = (mean_win - mean_multi) / var_P**0.5
deg_f = var_P ** 2 / ((var_win / n_win)**2 / (n_win - 1) + (var_multi / n_multi)**2 / (n_multi - 1))
p_val = ss.t.cdf(t_stat, df = deg_f) * 2

print("statistic = {} and p-value = {}".format(t_stat, p_val))

And we can see that we obtain a p-value of around 0.0008. Since this p-value is very low, we reject the null hypothesis and conclude that the two population means are significantly different.

Instead of doing all the calculation manually, which can be a tedious, we can also use the `statsmodels` package to take care of the computation for us and present us with the results (including additional calculations such as confidence intervals):

In [ ]:
def t_test(a, b, alpha = 0.05, alternative = 'two-sided', type = 'independent'):

    diff = a.mean() - b.mean()
    if type == 'independent':
        res = ss.ttest_ind(a, b, equal_var = False)
    elif type == 'paired':
        assert(len(a) == len(b))
        res = ss.ttest_rel(a, b)
      
    means = ws.CompareMeans(ws.DescrStatsW(a), ws.DescrStatsW(b))
    confint = means.tconfint_diff(alpha = alpha, alternative = alternative, usevar = 'unequal') 
    degfree = means.dof_satt()

    index = ['deg_of_freedom', 'observed_difference', 't-statistic', 'p_value', 'low_95_CI', 'high_95_CI']
    return pd.Series([degfree, diff, res[0], res[1], confint[0], confint[1]], index = index)   
   

t_test(windows, multi_platform)

Either way we get the same t-statistic and p-value. Examine these statistics noticing the following:

1. Difference in means is not enormous, 2.7 points on a 100 point scale, and the populations overlap quite a bit, but the t-statistic is large in absolute value and the p-value is small. Therefore, there is a small chance that the difference in means is from random variation alone. 
3. The 95% confidence interval does not straddle 0.  

Based on these statistics we can reject the null hypothesis and conclude that the two samples did not come from populations with the same mean.

## Statistical power analysis

The **power of a test** is formally defined $\text{power} = P(\text{reject } H_0| \text{ when } H_0 \text{ is false})$. In plain language, the power of a test is the probability of finding a true effect to be a significant effect. A test with insufficient power will not detect a significant effect even when it should. Clearly, we want the most powerful test we can find for the situation. Computing power can be a bit complex, and analytical solutions can be difficult or impossible. Often, a simulation is used to compute power.

Let's look at the example of computation power for the two sample t-tests for the difference of means. The power of this test depends on the several parameters:

- the number of samples
- the difference in the sample means standardized by their overall variability (also known as the **effect size**)
- the type of test

When running a power test, you can ask several questions that will assist you in designing an experiment. Usually, you will determine how big a sample you need to have a good chance of rejecting the null hypothesis. You can also determine how big an effect needs to be given a fixed sample size (all the samples you have or can afford) to have a good chance of rejecting the null hypothesis. 

The `statsmodels` package provides power calculations for a limited set of hypothesis tests. We can use these capabilities to examine the power. We can use it to find the right sample size for an effect size.


The `statsmodels` package provides power calculations for a limited set of hypothesis tests. We can use these capabilities to examine the power. We can use it to find the right sample size for a effect size.

In [ ]:
sample_size = np.arange(start = 2, stop = 12, step = 1)
power = [sp.tt_ind_solve_power(effect_size = abs(t_stat), nobs1 = n, alpha = 0.05, 
                               power = None, ratio = 1.0, alternative = 'two-sided') 
          for n in sample_size]

fig, ax = plt.subplots(1,1,figsize = (12,5))
sns.lineplot(x = sample_size, y = power, ax=  ax)
plt.title('power vs. sample size')
plt.xlabel('sample size')
plt.ylabel('power');

In practice, less than $80\%$ power for a test is not good. We can also use it to find the right effect size for a given sample size.

In [ ]:
effect_size = np.arange(start = 0.0, stop = 10.0, step = .2)
power = [sp.tt_ind_solve_power(effect_size = x, nobs1 = 25, alpha = 0.05, 
                               power = None, ratio = 1.0, alternative = 'two-sided') 
          for x in effect_size]

fig, ax = plt.subplots(1, 1, figsize =(10,5))
sns.lineplot(x = effect_size, y = power, ax = ax)
plt.axvline(abs(t_stat), 0, color = 'blue') # marks the observed effect
plt.title('power vs. effect size')
plt.xlabel('effect size')
plt.ylabel('power');

Examine these results and notice how the power of the t-test rapidly increases as the difference in means increases. At a relatively small difference in means the power of the test is approaching 1.0, the maximum possible. 

### Exercise 

- Find out if there is a significant difference in metacritic scores between platform distributions (`windows_only`) if we limit the data to the most popular games only: `popularity` == 2:

In [ ]:
windows_pop = scores.loc[((scores['windows_only'] == 1) & (scores['popularity'] == 2)), 'meta_score']
multi_platform_pop = scores.loc[((scores['windows_only'] == 0) & (scores['popularity'] == 2)), 'meta_score']
t_test(windows_pop,multi_platform_pop)

- Find out if there is a significant difference between user scores on IGDB (`igdb_uscore`) and on Steam, (`store_uscore`) for the most popular games (`popularity` == 2). HINT: For this you need to perform a paired t-test.

In [ ]:
## your code goes here

Since $P-value < \alpha$, we reject the null hypothesis and conclude that IGDB and Steam scores for popular games are significantly different from each other.

We saw that we can fail to reject the null hypothesis if the test lacks power. Let's investigate that.

- Plot 100 values for effect size ranging from $0.0$ to $10.0$ and measure the effect size required for the above test.

In [ ]:
## your code goes here

### End of exercise

## Tests for categorical data

The t-test is used on continuous data. What if we have categorical data?

### Chi-squared tests

We use Chi-squared (also written as $\chi^2)$ tests to compare counts in different categories, where the categories are mutually exclusive.

- A Chi-squared **goodness of fit** test (also called **test of proportions** or **Pearson's Chi-squared test**) is used to test if the sample is representative of the population. For example, if we survey people randomly in Washington State, we can test if the number of respondents in Seattle is significantly different from the expected probability based on Seattle's percentage of the state population.
- The Chi-squared **test of independence** (also called **test of association**) tests whether two categorical variables are related to each other. It does so by comparing observed counts to counts we should expect if the two variables are independent.

The name Chi-squared comes from a corresponding probability distribution. It's density depends on a parameter we call **degrees of freedom**, which is greater the bigger the sample size.

![](https://library.startlearninglabs.uw.edu/DATASCI410/img/Chi-square.png)

As with any probability density function, confidence intervals and p-values can be computed. Notice that the $\chi$-squared distribution becomes flatter and with greater dispersion as the degrees of freedom increase. In practice, this means that you will need large samples to get a meaningful result if you have too many choices in your test.

#### Chi-squared test of proportion example

Here's an example of the Chi-squared goodness of fit: The traditional way to apply a chi-squared test is to first create a chi-squared table. In this example we are looking at the results of an A-B test with three possible outcomes. This type of test might be applied to determine if a new web site drives more customer purchases.

The code in the cell below builds a simple chi-squared table. The data we have are

- actual occurrence of events
- expected probability of these events (based on the distribution of the null hypothesis)

From the above data we then calculate the following

- the expected occurrence of events given the expected probabilities
- the difference between the occurrence and the expected number of events
- the square of the difference
- the squared difference normalized by the expected number of occurrences

The sum of the figures in the last column is the Chi-squared statistic. In other words 

$$\chi^2_{\text{statistic}} = \sum_{i = 1}^k \frac{(o_i - e_i)^2}{e_i}$$ 

where we have $k$ groups, $o_i$ is the observed count for group $i$ and $e_i$ is the expected count for group $i$ based on the hypothesized proportions. Under $H_0$ we assume $\chi^2_{\text{statistic}} \sim \chi^2(k-1)$ where $k-1$ are the degrees of freedom.

In the complete steamdb dataset, proportion of games in the price brackets for `price` (1: < $19.99, 2: $19.99 - $24.99, 3: > $24.99)
is 88 percent for low, 8 percent for middle and 4 percent for high. Here are the counts we observe in the data:

In [ ]:
all_prices = [0.88, 0.08, 0.04]
price_counts = scores['price'].value_counts(sort = False)
price_counts

We can manually compute the $\chi^2$ statistic as such:

In [ ]:
price_data = pd.DataFrame({'price': ['low', 'middle', 'high'],
                         'observed_count': price_counts,
                         'hypothesized_proportion': all_prices})

total_count = price_data['observed_count'].sum()
price_data['expected_count'] = total_count * price_data['hypothesized_proportion']

price_data['diff_squared'] = (price_data['observed_count'] - price_data['expected_count'])**2
price_data['diff_normalized'] = price_data['diff_squared'] / price_data['expected_count']

chi_squared_statistic = price_data['diff_normalized'].sum()
print("chi_squared_statistic = {}".format(chi_squared_statistic))
price_data

We have $3-1 = 2$ degrees of freedom for the $\chi^2$ distribution. We can get a p-value for this statistic now:

In [ ]:
1 - ss.chi2.cdf(chi_squared_statistic, df = 2)

This p-value is rather small, so we reject the null hypothesis. In other words, assuming the hypothesized proportions for each group, there is little chance that the differences between the observed and expected occurrences are the result of random variation alone.

In the foregoing example we computed the chi-squared statistic and p-value directly. In general, this is a somewhat cumbersome approach. Instead, we can use the `chisquare` function from the `scipy.stats` package as shown in the code below.

In [ ]:
chi_sq_stat, p_value = ss.chisquare(price_data['observed_count'], price_data['expected_count'])
print('Chi-squared statistic = {} and p-value = {}.'.format(str(chi_sq_stat), str(p_value)))

Finally, we should check the power of our test. First we compute our observed effect size:

In [ ]:
eff_size = statsmodels.stats.gof.chisquare_effectsize(price_counts / price_counts.sum(), all_prices)
eff_size

We can see where we are by plotting effect size against power:

In [ ]:
effect_size = np.arange(start = 0.1, stop = 2.0, step = 0.1)
power = sp.GofChisquarePower().solve_power(effect_size = effect_size, nobs = price_counts.sum(), 
                                           n_bins = 3, alpha = 0.05)

fig, ax = plt.subplots(1, 1, figsize =(10,5))
sns.lineplot(x = effect_size, y = power, ax = ax)
plt.axvline(abs(eff_size), 0, color = 'blue') # marks the observed effect
plt.title('power vs. effect size')
plt.xlabel('effect size')
plt.ylabel('power');

Apparently, this test is quite powerful, and we could have detected much smaller differences between the observed and expected counts.

Let's perform a Chi-squared test of independence to see if two categorical variables are related or not. Let's see if the difficulty of a game is related to the time it takes to complete it

In [ ]:
ct = pd.crosstab(scores['time'], scores['difficulty'])
ct

In [ ]:
chi_sq_stat, p_value, dof, ct_expected = ss.chi2_contingency(ct)
print('Chi-squared statistic = {} and p-value = {}.'.format(str(chi_sq_stat), str(p_value)))

We can also calculate the effect size by flattening the observed and expected counts tables and turning counts into percentages.

In [ ]:
obs = ct.values.ravel() / ct.values.sum()
exp = ct_expected.ravel() / ct_expected.sum()

eff_size = statsmodels.stats.gof.chisquare_effectsize(obs, exp)
eff_size

If we return to the chart of power vs effect size we would see that this test has just enough power.

## Testing distributions

Tests of hypotheses often make distributional assumptions. This raises the question of how can we test if our data fits some distribution. To answer this question we will now look at both graphical and formal tests. Since in most cases, distribution assumptions are approximate, simple graphical methods are often sufficient.

### The Q-Q plot

The quantile-quantile (Q-Q) plot provides a handy visual means to inspect the similarity of distributions of a data set. The general idea is to plot the quantiles of the sample on the vertical axis and the quantiles of the theoretical distribution on the horizontal axis. If the points of the plot fall on an approximately straight line, you can conclude that the sample distribution is close to the theoretical. 

The normal Q-Q plot plots the quantiles of a standard normal distribution on the horizontal axis and the quantiles of the data sample on the vertical axis. If the sample is normal the data points will fall in a straight line. 

Run the code in the cell below to create Q-Q plots of the two samples, and examine the results.

In [ ]:
fig, axs = plt.subplots(ncols = 2, figsize = (12,5))
sm.qqplot(scores['igdb_uscore'], line = 's', ax = axs[0])
sm.qqplot(scores['meta_score'], line = 's', ax = axs[1]);

The line of points on both Q-Q plots is close to straight, but with significant deviations on the tails indicating less dispersion than a standard normal distribution.

We can also visualize the **cumulative density functions** (CDFs) of the two samples. To make things easier, here's a function that plots cumulative distributions for each sample.

In [ ]:
def plot_cums(dist1, dist2, labels = ['sample 1', 'sample 2']):
    plt.figure(figsize = (10, 6))
    p = np.arange(len(dist1)) / (len(dist1) - 1) # calculate the proportional values of samples

    data_sorted = np.sort(dist1) # sort the first data sample
    sns.lineplot(x = data_sorted, y = p)
    
    data_sorted = np.sort(dist2) # sort the second data sample:
    sns.lineplot(x = data_sorted, y = p)

    plt.title('empirical CDFs of two datasets')
    plt.xlabel('range of data')
    plt.ylabel('percentile')
    plt.legend(labels = labels)

    return None

In [ ]:
plot_cums(scores['meta_score'], scores['igdb_uscore'], labels = ['meta_score', 'igdb_uscore'])

Note that since IGDB and Metecritic scores are on the same scale we didn't need to rescale the data, but in general we need to rescale the data prior to comparing the distributions. The GameFAQ ratings are on a 5 point scale, so we can directly compare with the 100 point ratings by normalizing the columns. 

In [ ]:
store_norm = (scores['store_uscore'] - scores['store_uscore'].mean()) / scores['store_uscore'].std()
gfq_norm = (scores['gfq_rating'] - scores['gfq_rating'].mean()) / scores['gfq_rating'].std()

plot_cums(store_norm, gfq_norm, labels = ['Store User Score', 'GameFAQ Rating'])

Notice that, as you might expect, the two CDFs are quite similar, indicating that the two samples are drawn from the same distribution. But are the deviations significant? We can answer that more formally using the **Kolmogorov-Smirnov test**.

### The Kolmogorov-Smirnov test

We can test whether a sample follows a theoretical distribution or if two distributions are significantly different by using the Kolmogorov-Smirnov test. The K-S statistic is just the maximum vertical distance between the CDF of the sample and the CDF of the theoretical distribution, or the CDFs of two independent samples. Since it is based on a simple deviation, the K-S test can test departure from any hypothetical distribution, not just normal.

The **K-S statistic is the maximum vertical difference** between the two cumulative density functions. Based on this distance and the number of samples, the p-value for the K-S test is computed. It is important that the **samples must be standardized** before applying the K-S test.

In [ ]:
# compare user score on the Steam store with a standard normal
ss.kstest(store_norm, 'norm')

In [ ]:
# compare user score on the Steam store with GameFAQs user's scores
ss.kstest(store_norm, gfq_norm)

The p-value is small when compared with a standard normal, therefore we reject the null hypothesis that the data is normally distributed (with $\mu = 0$ and $\sigma = 1$). In other words, assuming that the data is normally distributed, a sample like ours is quite unusual.

The p-value when comparing the user ratings on two different platforms shown a much more similar distribution even though the raw scales were different. 

The K-S test is rather general, as it can be applied to test any distribution. However, this means that the **power** of this test is limited. The power of a test is the probability of rejecting a null hypothesis when the alternative is true. 

As is always the case with classical statistics, a more powerful test can be created by adopting more restrictive assumptions. For example, the Shapiro-Wilk test has greater power, but is specifically for normal distributions. 

## Testing multiple groups with ANOVA

So far, we have only looked at tests for comparing two samples. What if we have multiple groups and we wanted to compare their means? Why can’t we just do multiple two-sample t-tests for all pairs? Because doing so results in increased probability of accepting a false hypothesis; e.g., if we had 7 groups, there would be $7 \choose 2$ $=21$ pairs to test.  Recall that the $\alpha$ cutoff is the probability of committing a type-I error, so if $\alpha = 0.05$ for any one test then we have a 95% probability of rejecting $H_0$ correctly. But this probability drops as we perform more tests: for 21 tests it is $0.95^{21} \approx 0.34$!

This problem is known as the problem of **multiple comparisons** or **base rate fallacy**. With large numbers of groups, there is a high probability of getting a false positive (type I error). Several adjustments to the multiple comparisons problem have been proposed, such as the **Bonferroni correction**: instead of $\alpha$ we use $\alpha_b = \frac{\alpha}{k}$ (where $k$ is the number of groups) as our p-value threshold. This tends to be a conservative approach that won't work well if $k$ is very high. There are other approaches that we won't get into here.

Instead of trying to find out all possible significant effects like we do above, we can first perform a test to see if there are *any* significant effects, without necessarily knowing which:

- Null hypothesis: All groups are samples from the same population.
- Alternative hypothesis: At least one group has a statistically different mean.

This type of analysis is called “ANalysis Of VAriance”, or ANOVA for short. ANOVA is one of a large family of models used for **experimental design**.

Let's have a look at how we would perform the comparisons between the multiple groups of data. First, make data independence and normality assumptions about the groups. Let $k$ be the number of treatment groups, $n$ be the number of data points and SS be the sum of squares. We can split total sum of squares **SSTotal** into the sum of squared errors *between* treatments **SST** and the sum of squared errors *within* treatments **SSE**: SSTotal = SST + SSE. We can divide each by its corresponding degrees of freedom to get the mean squared error between treatments **MST** and the mean square error within treatments **MSE**. Finally we can compute the F-statistic and derive the p-value by referring to the [F distribution](https://en.wikipedia.org/wiki/F-distribution). 

We leave out the details of the exact calculations, and instead we summarize the results in an ANOVA table as follows:

|Type|Sum of Squares|degrees of freedom|Mean Square E|F|Significance|
|---|:---:|:---:|:---:|:---:|:---:|
|between groups|SST|$k-1$|SST / $k-1$|F-statistic| p-value|
|within groups|SSE|$n-k$|SSE / $n-k$|||
|total|SSTotal|$n-1$||||

Let's now see an example. Let's say we wanted to test if on average the Metacritic scores are different for games with degrees of difficulty.

In [ ]:
fig, ax = plt.subplots(nrows= 1, ncols=1, figsize=(10,5))
sns.boxplot(x = 'difficulty', y = 'meta_score', data = scores, ax= ax);

The plot shows variation between the distributions of the groups, but are any of these differences significant? To answer that we use an ANOVA test:

In [ ]:
model = ols('meta_score ~ C(difficulty)', data = scores).fit()
anova_table = sm.stats.anova_lm(model)
print(anova_table)

The F statistic is fairly large and the p-value is small. So we reject the null hypothesis and conclude that there are significant differences between the different groups.

Note that ANOVA isn't just a test, but also a statistical model. This means that once we fit it to the data, we can then use it to obtain predictions:

In [ ]:
model.predict({'difficulty': [1, 2, 3]}) # we get a unique prediction for each value of difficulty

Let's draw the distribution (histogram) of the actual scores:

In [ ]:
sns.displot(scores['meta_score'], 
            kde = False, 
            color = 'blue');

We can see that the predicted scores are less varied than the actual scores. In fact, the model predicts only three distinct scores. This should not come as a surprise, since we trained a model to predict the Metacritic score using **only** the `difficulty` rating from gamespot.com (ignoring all the other variables). So this model is returning a unique score for each difficulty group. We can make this model richer by adding more variables to it, but that's the topic of another lecture.

### Tukey's multiple comparison test

From the above ANOVA results we know that there is some difference in the means of these groups. However, ANOVA does not tell us which pairs of groups are significantly different from each other. From the box plot, we could make an educated guess, but one formal way of comparing groups is using **Tukey's multiple comparison** test. The test returns the following for each pairing of groups:

- the difference of the means
- the confidence interval of the difference in the means
- the p-value from the distribution of the differences

These results are laid out in a table or can be plotted graphically. Only differences in means with a confidence interval not overlapping zero are considered significant.

In [ ]:
Tukey_HSD = pairwise_tukeyhsd(scores['meta_score'], scores['difficulty'])
print(Tukey_HSD)

Examine the table above. If the difference in means between the variables is significant, the confidence interval will not include 0. Which pairs have a significant difference at the 95% confidence level? You can see the results of this test in the rightmost column of the table.

# Assignment

In this assignment, we want to reinforce the concepts we covered in the lecture. Let's first load the required libraries.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(rc = {'figure.figsize': (10, 8)})
import matplotlib.pyplot as plt

import scipy
import scipy.stats as ss
import numpy.random as nr
import statsmodels
import statsmodels.stats as st
import statsmodels.api as sm
import statsmodels.stats.power as sp
import statsmodels.stats.weightstats as ws
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols

We will be using the automobile mileage data for this assignment.

In [ ]:
def read_auto_data(file = "../data/canadian_cars_2022.csv"):
    'Function to load the auto data set from a .csv file' 

    ## Read the .csv file with the pandas read_csv method
    df = pd.read_csv(file)
    
    ## Split the number of gears from the type of transmission, decode fuel
    df['gears'] = df['transmission'].str.extract(r'([0-9]+)').astype('Int64')
    df['gears'] = df['gears'].fillna(1) # "gearless" continuously_variable vehicles
    df['fuel'].replace({'X': 'regular_gas', 
                             'Z': 'premium_gas', 
                             'D': 'diesel'}, inplace = True)
    df['transmission'] = df['transmission'].str.extract(r'([A-Z]+)')
    df['transmission'].replace({'A': 'automatic', 
                             'AM': 'automated_manual', 
                             'AS': 'automatic_select_shift', 
                             'AV': 'continuously_variable', 
                             'M': 'manual'}, inplace = True)
    
    ## Remove rows with missing values
    df = df.dropna(axis = 0).reset_index(drop= True)
    return df


auto_df = read_auto_data()
auto_df.head()

Run the following tests on the data:

1. Test whether `fuel_consumption_mpg` and log `fuel_consumption_mpg` (using `np.log10`) follow a normal distribution. Use both a **graphical** method and a **formal** test. For the rest of this exercise, choose between using mpg or log mpg based on which of the two best fits a normal distribution. <span style="color:red" float:right>[5 point]</span>

In [ ]:
## your code goes here

We can see from the QQ plots that the data appears to be right-tailed, with a few cars having very high mileage. Using a log transformation reduces the skew, but it also pushes the lowest mileages off the normal distribution curve as well.

2. Test if the fuel consumption is significantly different for the following populations of vehicles
- "Big 3" North American brands ('buick', 'cadillac', 'chevrolet', 'chrysler', 'dodge', 'ford',  'gmc', 'jeep', 'lincoln') compared with brands that began in other countries
- Vehicles with 1 gear vs many gears
- Vehicle with greater than median height vs less than or equal the median height
You are running separate tests for each variable. Use both graphical methods and the formal test. <span style="color:red" float:right>[5 point]</span>

In [ ]:
## your code goes here

3. Apply ANOVA and Tukey's HSD test to the miles per gallon to compare the fuel economy of autos for different vehicle classes. Restrict the analysis to just the `vehicle_class` categories having 10 or more cars in the data. Note that ANOVA and Tukey's HSD are **two separate tests**! <span style="color:red" float:right>[5 point]</span>

ANOVA tests whether there are any significant differences between any of the categories: $H_0: $ are categories have the same mean mpg, and $H_1: $ at least one category has a different mean mpg. If the p-value for ANOVA is significant, then we can perform a Tukey's HSD test to see which categories are significantly different from each other.

In [ ]:
## your code goes here

4. Graphically explore the differences in mileage of the cars with different body styles. If any of these relationships are statistically significant (as suggested by Tukey's HSD), examine the sample size and decide if they should be considered practically significant. <span style="color:red" float:right>[5 point]</span>

In [ ]:
## your code goes here

   
Note that to get full grade, for graphical tests you should include commentary on what your plot is showing. For formal tests should include the following:
- begin by naming the test you are using
- begin by clearly stating the null and alternative hypotheses
- run the test and report the statistic and p-value
- based on the p-value you should state the conclusion

# End of assignment